In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Install dependencies**

In [1]:
!pip install -q -U transformers accelerate bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 58.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.9 MB/s eta 0:00:00


**Logs into hugging face.**

In [1]:
import os
import gc
import random
import io
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import transformers
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gradio as gr

from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

# --- CONFIGURATION & AUTHENTICATION ---
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)
transformers.logging.set_verbosity_error()

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN") 
    login(token=hf_token)
    print(f"✅ Successfully Logged into Hugging Face! Pipeline Online ({DEVICE})")
except Exception as e:
    print("⚠️ Could not log in. Ensure your Kaggle Secret is named 'HF_TOKEN'.")

⚠️ Could not log in. Ensure your Kaggle Secret is named 'HF_TOKEN'.


**Neuro-Symbolic Architecture & Generators** : It defines the Physics engine and biological distributions. 

In [ ]:
# =================================================================
# ADVANCED HDR PREPROCESSING & DATA GENERATION
# =================================================================
def dynamic_scaling_advanced(signal_tensor, window_size=51):
    pad = window_size // 2
    local_max = F.max_pool1d(signal_tensor, kernel_size=window_size, stride=1, padding=pad)
    local_min = -F.max_pool1d(-signal_tensor, kernel_size=window_size, stride=1, padding=pad)
    
    global_max = signal_tensor.max(dim=-1, keepdim=True)[0]
    noise_floor = torch.clamp(global_max * 0.02, min=1e-6)
    
    raw_contrast = local_max - local_min
    local_contrast = torch.maximum(raw_contrast, noise_floor)
    return (signal_tensor - local_min) / local_contrast

class CKDMetabolomeGenerator:
    def __init__(self, seed=42):
        self.rng = np.random.RandomState(seed)
        self.ppm_axis = np.linspace(0, 10, 1000)
    
    def pseudo_voigt(self, x, center, amp, width, eta=0.6):
        sigma = width / (2 * np.sqrt(2 * np.log(2)))
        gamma = width / 2
        g = np.exp(-(x - center)**2 / (2 * sigma**2 + 1e-8))
        l = 1 / (1 + ((x - center) / (gamma + 1e-8))**2)
        return amp * (eta * l + (1 - eta) * g)

    def generate_batch(self, n_samples=100):
        inputs, targets_cls, targets_reg, labels = [], [], [], []
        
        for _ in range(n_samples):
            is_sick = self.rng.choice([0, 1])
            ph_shift = self.rng.uniform(-0.03, 0.03) 
            cr_amp = self.rng.uniform(0.8, 1.2)
            
            if is_sick:
                cit_ratio = np.clip(self.rng.normal(0.4, 0.2), 0.1, 0.8)   
                lac_ratio = np.clip(self.rng.normal(1.2, 0.3), 0.7, 2.0)   
                tmao_ratio = np.clip(self.rng.normal(1.0, 0.3), 0.5, 1.8)  
                tau_ratio = np.clip(self.rng.normal(0.2, 0.1), 0.05, 0.5)  
            else:
                cit_ratio = np.clip(self.rng.normal(1.0, 0.3), 0.5, 1.8)   
                lac_ratio = np.clip(self.rng.normal(0.4, 0.2), 0.1, 0.9)   
                tmao_ratio = np.clip(self.rng.normal(0.3, 0.2), 0.0, 0.8)  
                tau_ratio = np.clip(self.rng.normal(0.8, 0.2), 0.4, 1.2)   
            
            sig_cr = self.pseudo_voigt(self.ppm_axis, 3.05 + ph_shift, cr_amp, 0.03) + \
                     self.pseudo_voigt(self.ppm_axis, 4.05 + ph_shift, cr_amp*0.8, 0.03)
            sig_cit = self.pseudo_voigt(self.ppm_axis, 2.55 + ph_shift, cr_amp * cit_ratio, 0.04) + \
                      self.pseudo_voigt(self.ppm_axis, 2.68 + ph_shift, cr_amp * cit_ratio, 0.04)
            sig_lac = self.pseudo_voigt(self.ppm_axis, 1.33 + ph_shift, cr_amp * lac_ratio, 0.035)
            sig_tmao = self.pseudo_voigt(self.ppm_axis, 3.26 + ph_shift, cr_amp * tmao_ratio, 0.03)
            sig_tau = self.pseudo_voigt(self.ppm_axis, 3.30 + ph_shift, cr_amp * tau_ratio, 0.05) + \
                      self.pseudo_voigt(self.ppm_axis, 3.42 + ph_shift, cr_amp * tau_ratio, 0.05)

            n_decoys = self.rng.randint(5, 15) 
            decoy_signal = np.zeros_like(self.ppm_axis)
            exclusion_zones = [(2.45, 2.75), (1.20, 1.50), (2.95, 3.15), (3.95, 4.15), (3.20, 3.32), (3.26, 3.48)]
            
            for _ in range(n_decoys):
                for attempt in range(20):
                    decoy_ppm = self.rng.uniform(0.5, 9.5)
                    in_exclusion = any(lo <= decoy_ppm <= hi for lo, hi in exclusion_zones)
                    if not in_exclusion: break
                decoy_amp = self.rng.uniform(0.05, 0.15) 
                decoy_width = self.rng.uniform(0.02, 0.06)
                decoy_signal += self.pseudo_voigt(self.ppm_axis, decoy_ppm, decoy_amp, decoy_width)
            
            is_flat_baseline = self.rng.rand() > 0.5
            if is_flat_baseline:
                protein_baseline = np.zeros_like(self.ppm_axis)
            else:
                protein_baseline = 0.5 * np.sin(self.ppm_axis * 0.8) if is_sick else 0.1 * np.sin(self.ppm_axis * 0.5)

            raw_mix = sig_cr + sig_cit + sig_lac + sig_tmao + sig_tau + decoy_signal + protein_baseline
            phase = self.rng.uniform(-0.1, 0.1)
            dispersion = np.imag(scipy.signal.hilbert(raw_mix))
            distorted = raw_mix * np.cos(phase) + dispersion * np.sin(phase)
            
            is_qc_fail = self.rng.rand() < 0.1 
            if is_qc_fail: distorted = self.rng.normal(0, 0.5, 1000) 
            noisy_input = distorted + self.rng.normal(0, 0.02, 1000)

            t_cls, t_reg = np.zeros((5, 1000)), np.zeros((5, 1000))
            signals = [sig_cr, sig_cit, sig_lac, sig_tmao, sig_tau]
            for c, sig in enumerate(signals):
                if not is_qc_fail:
                    mask = sig > (0.1 * sig.max() + 1e-6)
                    t_cls[c, mask] = 1.0
                    t_reg[c] = sig 

            inputs.append(noisy_input); targets_cls.append(t_cls); targets_reg.append(t_reg); labels.append(is_sick)
            
        return np.array(inputs), np.array(targets_cls), np.array(targets_reg), np.array(labels), self.ppm_axis

# =================================================================
# DEEP LEARNING ARCHITECTURE
# =================================================================
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels//4, kernel_size=1)
        self.branch2 = nn.Conv1d(in_channels, out_channels//4, kernel_size=3, padding=1)
        self.branch3 = nn.Conv1d(in_channels, out_channels//4, kernel_size=5, padding=2)
        self.branch4 = nn.Conv1d(in_channels, out_channels//4, kernel_size=7, padding=3)
        self.bn = nn.BatchNorm1d(out_channels)
    def forward(self, x):
        out = torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)
        return F.leaky_relu(self.bn(out), 0.1)

class SanjeevaniEngine(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc1 = InceptionBlock(1, 32)
        self.pool = nn.MaxPool1d(2)
        self.enc2 = InceptionBlock(32, 64)
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.decoders = nn.ModuleList([
            nn.Sequential(
                nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
                nn.BatchNorm1d(32),
                nn.LeakyReLU(0.1)
            ) for _ in range(5)
        ])
        self.heads_cls = nn.ModuleList([nn.ConvTranspose1d(32, 1, kernel_size=4, stride=2, padding=1) for _ in range(5)])
        self.heads_reg = nn.ModuleList([nn.ConvTranspose1d(32, 1, kernel_size=4, stride=2, padding=1) for _ in range(5)])
        
    def forward(self, x):
        x = self.pool(self.enc1(x))
        x = self.pool(self.enc2(x))
        x = x.permute(0, 2, 1)
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        cls_outputs, reg_outputs = [], []
        for i in range(5):
            dec = self.decoders[i](x)
            cls_outputs.append(self.heads_cls[i](dec))
            reg_outputs.append(F.relu(self.heads_reg[i](dec)))
        return torch.cat(cls_outputs, dim=1), torch.cat(reg_outputs, dim=1)

**Training the Physics Engine**

In [ ]:
print("\n🚀 Training Custom Physics Engine (Sanjeevani V1)...")

gen = CKDMetabolomeGenerator(seed=42)
X, Y_cls, Y_reg, Y_labels, ppm = gen.generate_batch(10000)

X_t = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(DEVICE)
Y_cls_t = torch.tensor(Y_cls, dtype=torch.float32).to(DEVICE)
Y_reg_t = torch.tensor(Y_reg, dtype=torch.float32).to(DEVICE)
X_t = dynamic_scaling_advanced(X_t)

model = SanjeevaniEngine().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

crit_cls = nn.BCEWithLogitsLoss(reduction='none') 
crit_reg = nn.MSELoss(reduction='none')
channel_weights = torch.tensor([1.0, 2.0, 3.0, 5.0, 2.0], dtype=torch.float32).to(DEVICE)
w = channel_weights.view(1, 5, 1)

model.train()
BATCH_SIZE = 64
n_batches = len(X) // BATCH_SIZE

for epoch in range(25):
    epoch_loss = 0
    indices = torch.randperm(len(X))
    for i in range(n_batches):
        batch_idx = indices[i*BATCH_SIZE : (i+1)*BATCH_SIZE]
        optimizer.zero_grad()
        pred_cls, pred_reg = model(X_t[batch_idx])
        loss_cls = (crit_cls(pred_cls, Y_cls_t[batch_idx]) * w).mean()
        loss_reg = (crit_reg(pred_reg, Y_reg_t[batch_idx]) * w).mean()
        loss = loss_cls + loss_reg
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch+1)%5==0: 
        print(f"   Epoch {epoch+1}: Loss {epoch_loss/n_batches:.4f}")

torch.save(model.state_dict(), "sanjeevani_engine.pth")
del model, optimizer, X, Y_cls, Y_reg, X_t, Y_cls_t, Y_reg_t
gc.collect(); torch.cuda.empty_cache()
print("💾 Physics Engine weights saved successfully.")

**Loading HAI-DEF Agents**: It safely spreads your VLMs and LLMs across the two T4 GPUs to prevent crashing.

In [ ]:
print("🧹 Aggressive RAM Clearance...")
gc.collect()
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

print("⏳ Loading Physics Engine to GPU 0...")
model = SanjeevaniEngine().to("cuda:0") 
model.load_state_dict(torch.load("sanjeevani_engine.pth"))
model.eval()

print("⏳ Loading Vision Agent (PaliGemma) onto GPU 0...")
pg_processor = AutoProcessor.from_pretrained("google/paligemma-3b-mix-224")
pg_model = PaliGemmaForConditionalGeneration.from_pretrained(
    "google/paligemma-3b-mix-224",
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True
)

gc.collect() 

print("⏳ Loading Clinical Agent (MedGemma) onto GPU 1...")
tokenizer = AutoTokenizer.from_pretrained("google/medgemma-1.5-4b-it")
clinical_model = AutoModelForCausalLM.from_pretrained(
    "google/medgemma-1.5-4b-it",
    quantization_config=bnb_config,
    device_map={"": 1},
    low_cpu_mem_usage=True
)

print("✅ ALL MODELS LOADED SUCCESSFULLY AND READY FOR UI!")

**Launching Sanjeevani UI**: This launches the fully deterministic Gradio app.

In [ ]:
print("🚀 Initializing Sanjeevani Clinical Dashboard...")

demo_cases = {}
gen_ui = CKDMetabolomeGenerator(seed=42)
X_pool, _, _, Y_pool, ppm = gen_ui.generate_batch(100) 

for i in range(100):
    if Y_pool[i] == 0 and X_pool[i].max() > 0.7:
        demo_cases["Patient A (Healthy Routine Screening)"] = {
            "signal": X_pool[i], 
            "profile": "45-year-old male. Routine rural screening. No prior history of kidney disease. Blood pressure normal."
        }
        break

for i in range(100):
    if Y_pool[i] == 1 and X_pool[i].max() > 0.7:
        demo_cases["Patient B (Type 2 Diabetic - High Risk)"] = {
            "signal": X_pool[i], 
            "profile": "62-year-old female. Type 2 Diabetes for 10 years. Complains of fatigue. Recent eGFR shows mild decline."
        }
        break

noise_signal = np.random.normal(0, 0.2, 1000)
demo_cases["Patient C (Degraded Sample / Noise)"] = {
    "signal": noise_signal, 
    "profile": "70-year-old male. Urine sample delayed in transit for 48 hours without refrigeration."
}

def run_sanjeevani_pipeline(patient_selection):
    case_data = demo_cases[patient_selection]
    signal_real = case_data["signal"]
    profile = case_data["profile"]
    
    # --- STAGE 1: PALI-GEMMA VISION QC ---
    plt.figure(figsize=(4, 4), dpi=100)
    plt.plot(ppm, signal_real, color='black', linewidth=1)
    plt.axvspan(3.0, 4.1, color='green', alpha=0.3) 
    plt.xlim(2.5, 4.5); plt.ylim(signal_real.min() - 0.2, signal_real.max() + 0.2)
    plt.axis('off')
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()
    buf.seek(0)
    qc_img = Image.open(buf).convert("RGB")
    
    prompt = "<image>answer en classify the signal inside the green box: 'sharp peaks' or 'flat noise'?"
    inputs = pg_processor(text=prompt, images=qc_img, return_tensors="pt").to(pg_model.device)
    with torch.no_grad():
        gen_ids = pg_model.generate(**inputs, max_new_tokens=10)
        finding = pg_processor.decode(gen_ids[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip().lower()
    
    is_qc_failed = "noise" in finding
    
    # --- STAGE 2: SANJEEVANI PHYSICS ENGINE ---
    x_tensor = torch.tensor(signal_real, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to("cuda:0")
    with torch.no_grad():
        _, p_reg = model(dynamic_scaling_advanced(x_tensor))
        reg = p_reg.cpu().squeeze().numpy()
        
    cr_amp_raw = reg[0, 300:415].max()
    cit_amp_raw = reg[1, 245:280].max()
    lac_amp_raw = reg[2, 125:145].max()
    tmao_amp_raw = reg[3, 315:335].max()
    
    cr_anchor = cr_amp_raw + 1e-6 
    cit_ratio = cit_amp_raw / cr_anchor
    lac_ratio = lac_amp_raw / cr_anchor
    tmao_ratio = tmao_amp_raw / cr_anchor

    cit_deficit = max(0, 1.0 - cit_ratio) 
    risk_score = (cit_deficit * 1.5) + (tmao_ratio * 1.0) + (lac_ratio * 0.8)

    CR_THRESHOLD = 0.01 
    if is_qc_failed or cr_amp_raw < CR_THRESHOLD or (lac_ratio == 0.0 and tmao_ratio == 0.0):
        pipeline_status = "❌ INCONCLUSIVE (QC FAILURE)"
        ratios_display = "Biomarker Math Halted: Data integrity compromised."
    elif risk_score > 1.0:
        pipeline_status = "⚠️ HIGH RISK (SICK DETECTED)"
        ratios_display = f"**Risk Score:** {risk_score:.2f} (Threshold: 1.0)\n* Citrate Ratio: {cit_ratio:.2f}\n* TMAO Ratio: {tmao_ratio:.2f}\n* Lactate Ratio: {lac_ratio:.2f}"
    else:
        pipeline_status = "✅ LOW RISK (HEALTHY)"
        ratios_display = f"**Risk Score:** {risk_score:.2f} (Threshold: 1.0)\n* Citrate Ratio: {cit_ratio:.2f}\n* TMAO Ratio: {tmao_ratio:.2f}\n* Lactate Ratio: {lac_ratio:.2f}"

    # --- STAGE 3: MEDGEMMA SYNTHESIS ---
    if "INCONCLUSIVE" in pipeline_status:
        doc_text = "ERROR: Sample quality check failed. The PaliGemma Vision Agent or the Mathematical Anchor Gate detected missing anchor peaks or extreme baseline distortion."
        patient_text = "Drafting halted. Please request a new sample."
    else:
        cit_status = "Depleted (Abnormal)" if cit_ratio < 0.8 else "Normal"
        # tmao_status = "Elevated (Abnormal)" if tmao_ratio > 0.5 else "Normal"
        lac_status = "Elevated (Abnormal)" if lac_ratio > 0.6 else "Normal"

        # Handle the non-detection edge case
        if tmao_ratio < 0.05:
            tmao_status = "Not Detected (Exclude from clinical reasoning)"
        elif tmao_ratio > 0.5:
            tmao_status = "Elevated (Abnormal)"
        else:
            tmao_status = "Normal"
        doc_prompt = f"""You are an expert Nephrologist. Review the following urinary NMR deconvolution metrics. Do not write code.
Patient Profile: {profile}

Biomarker Status:
- Citrate Ratio: {cit_ratio:.2f} ({cit_status})
- TMAO Ratio: {tmao_ratio:.2f} ({tmao_status})
- Lactate Ratio: {lac_ratio:.2f} ({lac_status})

Task: Write a concise, professional Clinical Assessment and Plan in a single short paragraph (maximum 4 sentences). Do NOT use bullet points or lists.
- Explicitly state which biomarkers are abnormal and explain their pathophysiological implications in the context of the patient's history.
- Conclude with a specific clinical next step.
Assessment and Plan:"""
        
        inputs_doc = tokenizer(doc_prompt, return_tensors="pt").to("cuda:1")
        with torch.no_grad():
            out_doc = clinical_model.generate(**inputs_doc, max_new_tokens=400) 
            doc_text = tokenizer.decode(out_doc[0], skip_special_tokens=True).replace(doc_prompt, "").strip()
            doc_text = doc_text.split("Critique")[0].split("Note:")[0].strip()

        pt_prompt = f"""You are a compassionate doctor messaging this patient directly through a secure hospital portal. Do not write code.
Patient History: "{profile}"

Task: Translate the following Doctor's Assessment and Plan into a short, warm, and natural-sounding message (3-4 sentences) to the patient.
- Address them directly ("You").
- Accurately reflect the exact level of concern and the specific next steps recommended in the Doctor's note.
- Do NOT mention specific chemicals, metabolites, numbers, or use repetitive phrasing.

Doctor's Assessment and Plan to Translate:
{doc_text}

Message:"""
        
        inputs_pt = tokenizer(pt_prompt, return_tensors="pt").to("cuda:1")
        with torch.no_grad():
            out_pt = clinical_model.generate(**inputs_pt, max_new_tokens=150) 
            patient_text = tokenizer.decode(out_pt[0], skip_special_tokens=True).replace(pt_prompt, "").strip()
            patient_text = patient_text.split("Critique")[0].split("Note:")[0].strip()

    return qc_img, f"**VLM Agent Output:** '{finding}'", pipeline_status, ratios_display, doc_text, patient_text

# --- GRADIO UI LAYOUT ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏥 Sanjeevani Clinical Dashboard")
    gr.Markdown("Automated CKD Triage and EMR Auto-Drafting via Neuro-Symbolic AI")
    
    with gr.Row():
        patient_dropdown = gr.Dropdown(choices=list(demo_cases.keys()), label="Select Incoming Patient File", value=list(demo_cases.keys())[0], scale=3)
        submit_btn = gr.Button("▶️ Execute Sanjeevani Pipeline", variant="primary", scale=1)
    
    with gr.Tabs():
        with gr.TabItem("🔬 1. Lab Diagnostics (AI Physics)"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### PaliGemma Vision Gatekeeper")
                    img_output = gr.Image(label="Anchor Region Extract (2.5-4.5 ppm)", height=300)
                    vlm_output = gr.Markdown()
                with gr.Column():
                    gr.Markdown("### Neuro-Symbolic Math Engine")
                    status_output = gr.Markdown("## Awaiting execution...")
                    ratios_output = gr.Markdown()
                    
        with gr.TabItem("🩺 2. Clinical Co-Pilot (Physician View)"):
            gr.Markdown("### MedGemma Synthesis")
            gr.Markdown("*Automated synthesis of mathematical ratios and patient history into a standard Assessment & Plan.*")
            doc_output = gr.Textbox(label="Generated Clinical Notes", lines=6)
            
        with gr.TabItem("📱 3. EMR Export (Patient Portal Draft)"):
            gr.Markdown("### MyChart / Patient Portal Auto-Draft")
            gr.Markdown("*MedGemma translation of the clinical notes for direct export to patient communication systems.*")
            patient_output = gr.Textbox(label="Patient-Friendly Message Draft", lines=6)

    submit_btn.click(
        fn=run_sanjeevani_pipeline,
        inputs=patient_dropdown,
        outputs=[img_output, vlm_output, status_output, ratios_output, doc_output, patient_output]
    )

demo.launch(share=True)